# gridgeo tour

In [ ]:
import pooch

fname = "ugrid.nc"
url = f"https://github.com/pyoceans/gridgeo/raw/refs/heads/main/tests/data/{fname}"

fname = pooch.retrieve(
    url,
    known_hash="sha256:4588e39f63e4c832f232cca42ff1e9eaec0428de46ff007482ee57cb3e03c285",
)

In [ ]:
import gridgeo

grid = gridgeo.GridGeo(fname, standard_name="sea_water_potential_temperature")

In [ ]:
import fiona

schema = {
    "geometry": "MultiPolygon",
    "properties": {"name": f"str:{len(grid.mesh)}"},
}
with fiona.open("grid.shp", "w", "ESRI Shapefile", schema) as f:
    f.write({"geometry": grid.__geo_interface__, "properties": {"name": grid.mesh}})

Methods

In [ ]:
[s for s in dir(grid) if not s.startswith("_")]

In [ ]:
grid.mesh

In [ ]:
print(f"The grid has {len(grid.polygons)} polygons, showing the first 5.")

grid.geometry.geoms[:5]

In [ ]:
grid.outline

Displaying all the polygons as vectors can be costly and crash jupyter! Let's make a raster representation of them using cartopy instead.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={"projection": ccrs.PlateCarree()})

kw = {
    "linestyle": "-",
    "alpha": 0.25,
    "color": "darkgray",
}
ax.triplot(grid.triang, **kw)
ax.coastlines(resolution="10m");

`to_geojson()` method returns a styled geojson-like `dict`

See https://github.com/mapbox/simplestyle-spec/tree/master/1.1.0 for styling options.

In [ ]:
kw = {
    "fill": "#fd7d11",
    "fill_opacity": 0.2,
    "stroke_opacity": 1,
    "float_precision": 2,
}

geojson = grid.to_geojson(**kw)
geojson["properties"]

or just use the `__geo_interface__`.

In [ ]:
grid.__geo_interface__.keys()

Saving the grid to as geojson file

In [ ]:
grid.save("grid.geojson", **kw)

shapefile

In [ ]:
grid.save("grid.shp")

or just plot using folium ;-)

In [ ]:
import folium

x, y = grid.outline.centroid.xy

m = folium.Map(location=[y[0], x[0]])
folium.GeoJson(grid.outline.__geo_interface__).add_to(m)
m.fit_bounds(m.get_bounds())

m